In [ ]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./Landslide2.csv"
device = "101"
save_path = f"./data/dev{device}_prepared.csv"
#save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_24476\3950661409.py:13: DtypeWarning: Columns (9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
0,2025-06-27 18:59:34.950000,101.0,41.500000,0.0,28.444725,79.391396,12.150635,8.638015,99.898972
2,2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
4,2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
6,2025-05-02 11:42:28.667259,101.0,28.900000,0.0,34.250019,62.839169,15.995875,8.637998,99.899010
8,2025-05-02 11:42:40.117564,101.0,28.799999,0.0,34.185932,62.661785,19.812029,8.637998,99.899010
...,...,...,...,...,...,...,...,...,...
308512,2025-06-29 20:20:49.190000,101.0,27.900000,0.0,29.483482,80.398491,8.771909,8.638011,99.898972
308515,2025-06-29 20:21:12.230000,101.0,27.900000,0.0,29.528877,80.196312,8.352998,8.638011,99.898972
308536,2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972
308539,2025-06-29 20:26:02.150000,101.0,27.900000,0.0,29.050890,81.302589,8.617684,8.638011,99.898972


In [23]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,149607,149607.0,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000
mean,2025-06-04 12:12:35.064885248,101.0,48.239375,0.220004,29.755443,75.709608,11.315496,8.638010,99.898969
min,2025-05-02 11:42:06.635276,101.0,27.299999,0.000000,24.279013,46.977417,0.000000,8.637940,99.166664
25%,2025-05-29 17:45:20.274356992,101.0,44.400002,0.000000,27.405968,70.947433,5.311314,8.638001,99.898964
50%,2025-06-10 20:02:30.664009984,101.0,49.000000,0.000000,28.474098,79.242622,11.374877,8.638014,99.898972
75%,2025-06-17 11:49:32.409999872,101.0,55.400002,0.000000,32.319370,80.932556,15.343357,8.638022,99.898987
max,2025-06-29 20:26:13.090000,101.0,66.599998,41.630600,38.586636,87.270775,32.062466,8.638072,99.899498
std,NaN,0.0,9.134768,1.692000,3.059886,7.981286,6.957680,0.000019,0.001893


In [24]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
#df = df.sort_values(['devID', 'timestamp']).reset_index(drop=True)

# ตั้ง timestamp เป็น index
df = df.set_index('timestamp')
df

,devID,soil,rain,temp,humi,geo,lat,lng
timestamp,,,,,,,,
2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
2025-05-02 11:42:28.667259,101.0,28.900000,0.0,34.250019,62.839169,15.995875,8.637998,99.899010
2025-05-02 11:42:40.117564,101.0,28.799999,0.0,34.185932,62.661785,19.812029,8.637998,99.899010
2025-05-02 11:42:50.732852,101.0,28.900000,0.0,34.175251,63.481953,8.099698,8.637998,99.899010
...,...,...,...,...,...,...,...,...
2025-06-29 20:20:49.190000,101.0,27.900000,0.0,29.483482,80.398491,8.771909,8.638011,99.898972
2025-06-29 20:21:12.230000,101.0,27.900000,0.0,29.528877,80.196312,8.352998,8.638011,99.898972
2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972


In [25]:
#df = df[(np.abs(zscore(df[['soil', 'rain', 'temp', 'humi', 'geo']])) < 20).all(axis=1)]
#df

In [26]:
'''
# ฟังก์ชันเติม NaN หลัง rolling
def fill_na(df_group):
    return df_group.fillna(method='bfill').fillna(method='ffill')

# กำหนด windows
rain_windows = {
    'rain_1h_sum': '1H',
    'rain_3h_sum': '3H',
    'rain_6h_sum': '6H',
    'rain_12h_sum': '12H',
    'rain_1d_sum': '1D',
    'rain_7d_sum': '7D',
}
soil_mean_windows = {
    'soil_1h_mean': '1H',
    'soil_3h_mean': '3H',
    'soil_6h_mean': '6H',
    'soil_12h_mean': '12H',
    'soil_1d_mean': '1D',
}
geo_mean_windows = {
    'geo_1h_mean': '1H',
    'geo_3h_mean': '3H',
    'geo_6h_mean': '6H',
    'geo_12h_mean': '12H',
    'geo_1d_mean': '1D',
}

# ต้อง reset_index เพื่อให้ devID เป็น column สำหรับ groupby
df_reset = df.reset_index()

# สร้าง group ตาม devID สำหรับ rolling
grouped = df_reset.groupby('devID')

# Rolling sums/means
for col_name, window in rain_windows.items():
    df_reset[col_name] = grouped['rain'].rolling(window, on='timestamp').sum().reset_index(level=0, drop=True).values
for col_name, window in soil_mean_windows.items():
    df_reset[col_name] = grouped['soil'].rolling(window, on='timestamp').mean().reset_index(level=0, drop=True).values
for col_name, window in geo_mean_windows.items():
    df_reset[col_name] = grouped['geo'].rolling(window, on='timestamp').mean().reset_index(level=0, drop=True).values

# เติม NaN ที่เกิดจาก rolling
df_reset = grouped.apply(fill_na)
df_reset.index = df_reset.index.droplevel(0)

df = df_reset  # อัปเดต df ให้เป็น DataFrame ที่เติม rolling แล้ว

df
df_reset.index = df_reset.index.droplevel(0)

df = df_reset  # อัปเดต df ให้เป็น DataFrame ที่เติม rolling แล้ว
'''

"\n# ฟังก์ชันเติม NaN หลัง rolling\ndef fill_na(df_group):\n    return df_group.fillna(method='bfill').fillna(method='ffill')\n\n# กำหนด windows\nrain_windows = {\n    'rain_1h_sum': '1H',\n    'rain_3h_sum': '3H',\n    'rain_6h_sum': '6H',\n    'rain_12h_sum': '12H',\n    'rain_1d_sum': '1D',\n    'rain_7d_sum': '7D',\n}\nsoil_mean_windows = {\n    'soil_1h_mean': '1H',\n    'soil_3h_mean': '3H',\n    'soil_6h_mean': '6H',\n    'soil_12h_mean': '12H',\n    'soil_1d_mean': '1D',\n}\ngeo_mean_windows = {\n    'geo_1h_mean': '1H',\n    'geo_3h_mean': '3H',\n    'geo_6h_mean': '6H',\n    'geo_12h_mean': '12H',\n    'geo_1d_mean': '1D',\n}\n\n# ต้อง reset_index เพื่อให้ devID เป็น column สำหรับ groupby\ndf_reset = df.reset_index()\n\n# สร้าง group ตาม devID สำหรับ rolling\ngrouped = df_reset.groupby('devID')\n\n# Rolling sums/means\nfor col_name, window in rain_windows.items():\n    df_reset[col_name] = grouped['rain'].rolling(window, on='timestamp').sum().reset_index(level=0, drop=True).v

In [27]:
"""
# ความต่างและ STD
df['soil_diff'] = grouped['soil'].diff().reset_index(level=0, drop=True)
df['geo_diff'] = grouped['geo'].diff().reset_index(level=0, drop=True)
df['soil_1h_std'] = grouped['soil'].rolling('1H').std().reset_index(level=0, drop=True)
df['geo_1h_std'] = grouped['geo'].rolling('1H').std().reset_index(level=0, drop=True)

# เติม NaN ที่เกิดจาก rolling
df = grouped.apply(fill_na)
df.index = df.index.droplevel(0)
"""
df = df.reset_index()  # เอา timestamp กลับมาเป็น column


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149607 entries, 0 to 149606
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  149607 non-null  datetime64[ns]
 1   devID      149607 non-null  float64       
 2   soil       149607 non-null  float64       
 3   rain       149607 non-null  float64       
 4   temp       149607 non-null  float64       
 5   humi       149607 non-null  float64       
 6   geo        149607 non-null  float64       
 7   lat        149607 non-null  float64       
 8   lng        149607 non-null  float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 10.3 MB


In [29]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [30]:

df.to_csv(f"{save_path}", index=False)

print("✅ Save")

✅ Save
